<a href="https://colab.research.google.com/github/TomasKricka/MPA-MLF/blob/main/pc8/Kafka_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Activation
from keras.optimizers import RMSprop

import numpy as np
import random
import pandas as pd


In [2]:
!wget -O kafka_Metamorphosis.txt https://www.gutenberg.org/files/5200/5200-0.txt

--2023-04-06 06:41:15--  https://www.gutenberg.org/files/5200/5200-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142017 (139K) [text/plain]
Saving to: ‘kafka_Metamorphosis.txt’

kafka_Metamorphosis 100%[===================>] 138.69K   816KB/s    in 0.2s    

2023-04-06 06:41:15 (816 KB/s) - ‘kafka_Metamorphosis.txt’ saved [142017/142017]



In [3]:
text = open('kafka_Metamorphosis.txt', 'r').read().lower()
print('text lenght: ', len(text))

text lenght:  138408


In [15]:
print(text[:1000])

﻿the project gutenberg ebook of metamorphosis, by franz kafka

this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restrictions
whatsoever. you may copy it, give it away or re-use it under the terms
of the project gutenberg license included with this ebook or online at
www.gutenberg.org. if you are not located in the united states, you
will have to check the laws of the country where you are located before
using this ebook.

** this is a copyrighted project gutenberg ebook, details below **
**     please follow the copyright guidelines in this file.     **

title: metamorphosis

author: franz kafka

translator: david wyllie

release date: may 13, 2002 [ebook #5200]
[most recently updated: may 20, 2012]

language: english

character set encoding: utf-8

copyright (c) 2002 by david wyllie.

*** start of the project gutenberg ebook metamorphosis ***




metamorphosis

by franz kafka

translated by david wyllie



In [4]:
chars = sorted(list(set(text)))
print('total chars: ' , len(chars))
print('total chars: ' , chars)

total chars:  62
total chars:  ['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ç', '—', '’', '“', '”', '\ufeff']


In [19]:
char_indicies = dict((c, i) for i, c in enumerate(chars))
indicies_char = dict((i, c) for i, c in enumerate(chars))

In [20]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])

print('nb sentences: ', len(sentences))

nb sentences:  46123


In [21]:
print(sentences[:3])
print(next_chars[:3])

['\ufeffthe project gutenberg ebook of metamorp', 'e project gutenberg ebook of metamorphos', 'roject gutenberg ebook of metamorphosis,']
['h', 'i', ' ']


In [32]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indicies[char]] = 1
    y[i, char_indicies[next_chars[i]]] = 1

In [36]:
model = Sequential()
model.add(LSTM(254, input_shape=(maxlen, len(chars))))
model.add(Dense(10*len(chars)))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [37]:
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(learning_rate = 0.01))

In [38]:
model.fit(x, y, batch_size = 128, epochs = 50)

Epoch 1/50
361/361 [==============================] - 5s 8ms/step - loss: 3.2047
Epoch 2/50
361/361 [==============================] - 3s 7ms/step - loss: 1.9309
Epoch 3/50
361/361 [==============================] - 3s 7ms/step - loss: 1.7068
Epoch 4/50
361/361 [==============================] - 3s 7ms/step - loss: 1.5582
Epoch 5/50
361/361 [==============================] - 3s 8ms/step - loss: 1.4369
Epoch 6/50
361/361 [==============================] - 3s 8ms/step - loss: 1.3327
Epoch 7/50
361/361 [==============================] - 3s 8ms/step - loss: 1.2337
Epoch 8/50
361/361 [==============================] - 3s 8ms/step - loss: 1.1412
Epoch 9/50
361/361 [==============================] - 3s 9ms/step - loss: 1.0599
Epoch 10/50
361/361 [==============================] - 3s 8ms/step - loss: 0.9800
Epoch 11/50
361/361 [==============================] - 3s 8ms/step - loss: 0.9090
Epoch 12/50
361/361 [==============================] - 3s 8ms/step - loss: 0.8374
Epoch 13/50
361/361 [====

In [55]:
def sample(preds, temperature = 1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

def generate_text(sentence, length, diversity):
  generated = ''
  generated += sentence
  for i in range(length):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_indicies[char]] = 1.

    preds = model.predict(x_pred, verbose = 0)[0]
    next_index = sample(preds, diversity)
    next_char = indicies_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char
  return generated


In [57]:
text = " the firts thing he wanted to do was to "
sentence = text[0: maxlen]

print(generate_text(sentence, 30,0.2))

 the firts thing he wanted to do was to do
rose himself, and then thou
